In [1]:
import pandas as pd
import nltk
from string import digits
from nltk.corpus import stopwords
import math
import numpy as np
import os
import sklearn
from sklearn.model_selection import KFold
import string
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import random

In [2]:
def file_name(file_dir): 
    for root, dirs, files in os.walk(file_dir):
        print(root) #当前目录路径
        print(dirs) #当前路径下所有子目录
        print(files) #当前路径下所有非目录子文件
        return files

In [3]:
files = file_name('..\data')

..\data
[]
['AlarmClock.txt', 'Baby.txt', 'Bag.txt', 'CableModem.txt', 'Dumbbell.txt', 'Flashlight.txt', 'Gloves.txt', 'GPS.txt', 'GraphicsCard.txt', 'Headphone.txt', 'HomeTheaterSystem.txt', 'Jewelry.txt', 'Keyboard.txt', 'Magazine_Subscriptions.txt', 'Movies_TV.txt', 'Projector.txt', 'RiceCooker.txt', 'Sandal.txt', 'Vacuum.txt', 'Video_Games.txt']


In [4]:
lamda = 1
alpha = 0.1
tau = 6
sigma = 6
learningRate = 0.1

In [5]:
def countFreq(review,dictName):
        for word in review.split():
            word = word.lower()
            try:
                eval(dictName).update({word : eval(dictName).get(word)+1})
            except:
                eval(dictName).update({word : 1})

In [6]:
domainNames = files.copy()
for i in range(20):
    domainNames[i] = files[i].split('.')[0]

In [14]:
for i in range(20):
    domainName = domainNames[i]
    currentData = pd.read_csv(('..\data\\')+files[i],sep='\t')
    dictName = domainName + 'PosFreq'
    exec( dictName + '= dict()')
    currentData[currentData['Label'] == 'POS']['Review'].apply(lambda X:countFreq(X,dictName))
    dictName = domainName + 'NegFreq'
    exec( dictName + '= dict()')
    currentData[currentData['Label'] == 'NEG']['Review'].apply(lambda X:countFreq(X,dictName))   

In [16]:
for i in range(20):
    domainName = domainNames[i]
    currentData = pd.read_csv(('..\data\\')+files[i],sep='\t')
    posFreqDict = eval(domainName + 'PosFreq')
    negFreqDict = eval(domainName + 'NegFreq')
    
    currentVocabulary = list(posFreqDict.keys()) + list(negFreqDict.keys())
    posProDict = dict()
    negProDict = dict()
    for word in currentVocabulary:
        freq = posFreqDict.get(word)
        if freq is None:
            freq = 0
        posProDict.update({word : (lamda + freq) / (lamda * len(currentVocabulary) + sum(posFreqDict.values())) })
        
    for word in currentVocabulary:
        freq = negFreqDict.get(word)
        if freq is None:
            freq = 0
        negProDict.update({word : (lamda + freq) / (lamda * len(currentVocabulary) + sum(negFreqDict.values())) })
        
    exec(domainName + 'PosPro = posProDict')
    exec(domainName + 'NegPro = negProDict')
    
    distiguishableWords = []
    for word in currentVocabulary:
        if posProDict.get(word)/negProDict.get(word) > sigma or posProDict.get(word)/negProDict.get(word) <1/6:
            distiguishableWords.append(word)
    exec(domainName + 'DistiguishableWords = distiguishableWords')         

In [17]:
def calculateProba(review,posPriorProb,negPriorProb,posFreq,negFreq,posProb,negProb,vocabularyNum):
    
    words = review.lower().split()
    
    posProba = math.log10(posPriorProb)
    negProba = math.log10(negPriorProb)
    posTotalFreq = sum(posFreq.values())
    negTotalFreq = sum(negFreq.values())
    for word in words:
        wordPosProba = posProb.get(word)
        if wordPosProba is None:
            posProba += math.log10( (lamda + 0)/(lamda*vocabularyNum + posTotalFreq)) ##
        else:
            if wordPosProba<0:
                pass
            else:
                posProba += math.log10(wordPosProba)
        
        
        wordNegProba = negProb.get(word)
        
        if wordNegProba is None:
            negProba += math.log10((lamda + 0)/(lamda*vocabularyNum + negTotalFreq))
        else:
            if wordNegProba<0:
                pass
            else:
                negProba += math.log10(wordNegProba)
    try:
        return (1-10**(negProba-posProba))/(1+10**(negProba-posProba)) ##normalize to -1,1
    except:
        if(negProba-posProba>20):
            return (1-10**int(negProba-posProba))/(1+10**int(negProba-posProba)) ##normalize to -1,1       

In [ ]:
scores =  np.zeros(21)
for k in range(20):
    f1Score = 0
    domians = files.copy()
    domians.remove(files[k])
    pastDomains = domainNames.copy()
    pastDomains.remove(domainNames[k])
    domainName = domainNames[k]
    distiguishableWords = eval(domainName + 'DistiguishableWords')
    
    
    histroyData = pd.read_csv(('..\data\\')+domians[0],sep='\t')
    for file in domians[1:-1]:
        histroyDataData = pd.concat([histroyData,pd.read_csv(('..\data\\')+file,sep='\t')])
    histroyData = histroyData[histroyData.apply(lambda X:X['Label']!='NEU',axis=1)]

    targetData = pd.read_csv(('..\data\\')+files[k],sep='\t')
    targetData = targetData[targetData.apply(lambda X:X['Label']!='NEU',axis=1)]
    
    histroyPosFreq = dict()
    histroyData[histroyData['Label'] == 'POS']['Review'].apply(lambda X:countFreq(X,'histroyPosFreq'))
    histroyNegFreq = dict()
    histroyData[histroyData['Label'] == 'NEG']['Review'].apply(lambda X:countFreq(X,'histroyNegFreq'))
    
    historyVocabulary = set(list(histroyPosFreq.keys()) + list(histroyNegFreq.keys()))
    historyPosMeta = dict()
    historyNegMeta = dict()
    reliableWords = []
    penaltyRatio = dict()
    
    for word in historyVocabulary:
        metaPosNum = historyPosMeta.get(word)
        metaNegNum = historyNegMeta.get(word)
        if metaPosNum is None:
            metaPosNum = 0
        if metaNegNum is None:
            metaNegNum = 0
        
        for currentDomain in pastDomains:
            temPosProDict = eval(currentDomain + 'PosPro')
            temNegProDict = eval(currentDomain + 'NegPro')
            temPosPro = temPosProDict.get(word)
            temNegPro = temNegProDict.get(word)
            if temPosPro is None:
                temPosPro = 0
            if temNegPro is None:
                temNegPro = 0
            if temPosPro > temNegPro:
                metaPosNum += 1
            if temPosPro < temNegPro:
                metaNegNum += 1
                
        historyPosMeta.update({word:metaPosNum})
        historyNegMeta.update({word:metaNegNum})
        if metaPosNum > tau or metaNegNum > tau:
            reliableWords.append(word)
            penaltyRatio.update({word: metaPosNum/(metaPosNum+metaNegNum)} )

    n=0
    kf = KFold(n_splits=5, random_state=None, shuffle=False)
    for train_index, test_index in kf.split(targetData):
        
        print('Fold ',n)
        trainData = targetData.iloc[train_index]
        testData = targetData.iloc[test_index]
        
        totalTrainData = pd.concat([histroyData,trainData])
        
        positiveNum = sum(totalTrainData['Label'] == 'POS')
        negativeNum = sum(totalTrainData['Label'] == 'NEG')
        totalTrainDataNum = len(totalTrainData['Label'])
        positivePro = positiveNum/totalTrainDataNum #posibility of the positive review in the training data
        negativePro = negativeNum/totalTrainDataNum #posibility of the negative review in the training data
        
        targetPosFreqDict = dict()
        trainData[trainData['Label'] == 'POS']['Review'].apply(lambda X:countFreq(X,'targetPosFreqDict'))
        targetNegFreqDict = dict()
        trainData[trainData['Label'] == 'NEG']['Review'].apply(lambda X:countFreq(X,'targetNegFreqDict'))
                                

        vocabulary = set(list(histroyPosFreq.keys()) + list(histroyNegFreq.keys()) + list(targetPosFreqDict.keys()) + list(targetNegFreqDict.keys()))
        targetVocabulary = set(list(targetPosFreqDict.keys()) + list(targetNegFreqDict.keys()))
                                    
        targetPosPro = dict()
        targetNegPro = dict()
                                    
        for word in targetVocabulary:
            freq = targetPosFreqDict.get(word)
            if freq is None:
                freq = 0
            targetPosPro.update({word : (lamda + freq) / (lamda * len(targetVocabulary) + sum(targetPosFreqDict.values())) })
        
        for word in targetVocabulary:
            freq = targetNegFreqDict.get(word)
            if freq is None:
                freq = 0
            targetNegPro.update({word : (lamda + freq) / (lamda * len(targetVocabulary) + sum(targetNegFreqDict.values())) })
                            
        distiguishableWords = []
        for word in targetVocabulary:
            if targetPosPro.get(word) is None or targetNegPro.get(word) is None:
                 pass
            else:
                if targetPosPro.get(word)/targetNegPro.get(word) > sigma or targetPosPro.get(word)/targetNegPro.get(word) <1/6:
                    distiguishableWords.append(word)
        
        posVirtualFreq = histroyPosFreq
        negVirtualFreq = histroyNegFreq
                                    
        for word in targetVocabulary:
            if posVirtualFreq.get(word) is None:
                posVirtualFreq.update({word : 0})
            if negVirtualFreq.get(word) is None:
                negVirtualFreq.update({word : 0})

        for review in trainData.iterrows():
            print('.',end="")
            review = review[1]
    
            if review['Label'] == 'POS':
                sent = review['Review'].lower()
                words = sent.split()


                term1 =negativePro/positivePro
                temNegFreq = negVirtualFreq.get(word)
                temPosFreq = posVirtualFreq.get(word)
                if temPosFreq is None:
                    temPosFreq = 0
                if temNegFreq is None:
                    temNegFreq = 0
                
                for word in words:
                    virturalFreq = posVirtualFreq.get(word)
                    if virturalFreq is None:
                        virturalFreq = 0
                    tagertDomainPenalty = 0
                    soruceDomainsPenalty = 0  
                    
                    try:
                        for word in set(words):
                            term1 *= ((lamda + temNegFreq)/(lamda + temPosFreq)) ** words.count(word)
                    except OverflowError as error:
                        term4 = len(words) /(lamda*len(vocabulary)+sum(posVirtualFreq.values() ))
                    else:
                        try:
                            term2 = ((lamda*len(vocabulary) + sum(posVirtualFreq.values())) / (lamda*len(vocabulary) + sum(negVirtualFreq.values())) ) ** len(words) 
                        except OverflowError as error:
                            term4 = len(words) / (lamda*len(vocabulary) + sum(negVirtualFreq.values()))
                        else:   
                            term3 = len(words)* term2 * 1/ (lamda*len(vocabulary) + sum(posVirtualFreq.values()))
                            term4 = (words.count(word)/(lamda+virturalFreq) + term1*term3)/(1 + term1 * term2)
                    if word in distiguishableWords:
                        tagertDomainPenalty = alpha * (virturalFreq - targetPosFreqDict.get(word, 0))
                    if word in reliableWords:
                        soruceDomainsPenalty = alpha * (virturalFreq - penaltyRatio.get(word)*histroyPosFreq.get(word, 0))
                    
                    updateValue = learningRate *(term4 -  words.count(word)/(lamda+virturalFreq) + tagertDomainPenalty + soruceDomainsPenalty)
                    if math.isnan(updateValue):
                        updateValue = 0
                    if virturalFreq + updateValue < 0:
                        updateValue = 0
                    posVirtualFreq.update({word :  virturalFreq + updateValue})
                    
        


                term1 =positivePro/negativePro    
                temNegFreq = negVirtualFreq.get(word)
                temPosFreq = posVirtualFreq.get(word)
                if temPosFreq is None:
                    temPosFreq = 0
                if temNegFreq is None:
                    temNegFreq = 0
                         

                for word in words:
                    virturalFreq = negVirtualFreq.get(word)
                    if virturalFreq is None:
                        virturalFreq = 0
                        
                    try:
                        for word in set(words):
                            term1 *= ((lamda + temPosFreq)/(lamda +  temNegFreq)) ** words.count(word)
                    except OverflowError as error:
                        term4 = 0 #虽然实际值不为0，且可计算。但由于值可能极其大，不建议更新。
                    else:
                        try:
                            term2 = ((lamda*len(vocabulary) + sum(posVirtualFreq.values())) / (lamda*len(vocabulary) + sum(negVirtualFreq.values())) ) ** len(words) 
                        except OverflowError as error:
                            term4 = 0
                        else:   
                            term3 = len(words)* term2 * (-1)/ (lamda*len(vocabulary) + sum(negVirtualFreq.values()))
                            term4 = (words.count(word)/(lamda+virturalFreq) * term2 + term3)/( term1 + term2)
                    if word in distiguishableWords:
                        tagertDomainPenalty = alpha * (virturalFreq - targetNegFreqDict.get(word, 0))
                    if word in reliableWords:
                        soruceDomainsPenalty = alpha * (virturalFreq - (1 - penaltyRatio.get(word)) * histroyNegFreq.get(word, 0))
                    updateValue =  learningRate * (term4 + tagertDomainPenalty + soruceDomainsPenalty)
                    if math.isnan(updateValue):
                        updateValue = 0
                    if virturalFreq + updateValue < 0:
                        updateValue = 0
                    negVirtualFreq.update({word : virturalFreq + updateValue})
            else: # 负文档
                term1 =negativePro/positivePro
                temNegFreq = negVirtualFreq.get(word)
                temPosFreq = posVirtualFreq.get(word)
                if temPosFreq is None:
                    temPosFreq = 0
                if temNegFreq is None:
                    temNegFreq = 0
                
                for word in words:
                    virturalFreq = posVirtualFreq.get(word)
                    if virturalFreq is None:
                        virturalFreq = 0
                    tagertDomainPenalty = 0
                    soruceDomainsPenalty = 0  
                    
                    try:
                        for word in set(words):
                            term1 *= ((lamda + temNegFreq)/(lamda + temPosFreq)) ** words.count(word)
                    except OverflowError as error:
                        term4 = 0
                    else:
                        try:
                            term2 = ((lamda*len(vocabulary) + sum(posVirtualFreq.values())) / (lamda*len(vocabulary) + sum(negVirtualFreq.values())) ) ** len(words) 
                        except OverflowError as error:
                            term4 = 0
                        else:   
                            term3 = len(words)* term2 * 1/ (lamda*len(vocabulary) + sum(posVirtualFreq.values()))
                            term4 = (words.count(word)/(lamda+virturalFreq) + term1*term3)/(1 + term1 * term2)
                    if word in distiguishableWords:
                        tagertDomainPenalty = alpha * (virturalFreq - targetPosFreqDict.get(word, 0))
                    if word in reliableWords:
                        soruceDomainsPenalty = alpha * (virturalFreq - penaltyRatio.get(word)*histroyPosFreq.get(word, 0))

                    updateValue =  learningRate *(term4 -  words.count(word)/(lamda+virturalFreq) + tagertDomainPenalty + soruceDomainsPenalty)
                    if math.isnan(updateValue):
                        updateValue = 0
                    if virturalFreq + updateValue < 0:
                        updateValue = 0
                    posVirtualFreq.update({word :  virturalFreq + updateValue})
                    
            


                term1 =positivePro/negativePro    
                temNegFreq = negVirtualFreq.get(word)
                temPosFreq = posVirtualFreq.get(word)
                if temPosFreq is None:
                    temPosFreq = 0
                if temNegFreq is None:
                    temNegFreq = 0
                         

                for word in words:
                    virturalFreq = negVirtualFreq.get(word)
                    if virturalFreq is None:
                        virturalFreq = 0
                        
                    try:
                        for word in set(words):
                            term1 *= ((lamda + temPosFreq)/(lamda +  temNegFreq)) ** words.count(word)
                    except OverflowError as error:
                        term4 = words.count(word)/(lamda + virturalFreq) - words.count(word)/(lamda*len(vocabulary) + sum(negVirtualFreq.values()))
                    else:
                        try:
                            term2 = ((lamda*len(vocabulary) + sum(posVirtualFreq.values())) / (lamda*len(vocabulary) + sum(negVirtualFreq.values())) ) ** len(words) 
                        except OverflowError as error:
                            term4 = 0 #虽然实际值不为0，且可计算。但由于值可能极其大，不建议更新。
                        else:   
                            term3 = len(words)* term2 * (-1)/ (lamda*len(vocabulary) + sum(negVirtualFreq.values()))
                            term4 = (words.count(word)/(lamda+virturalFreq) * term2 + term3)/( term1 + term2)
                    if word in distiguishableWords:
                        tagertDomainPenalty = alpha * (virturalFreq - targetNegFreqDict.get(word, 0))
                    if word in reliableWords:
                        soruceDomainsPenalty = alpha * (virturalFreq - (1 - penaltyRatio.get(word)) * histroyNegFreq.get(word, 0))
                        
                    updateValue =  learningRate * (term4 + tagertDomainPenalty + soruceDomainsPenalty)
                    if math.isnan(updateValue):
                        updateValue = 0
                    if virturalFreq + updateValue < 0:
                        updateValue = 0
                    negVirtualFreq.update({word :  virturalFreq + updateValue})
                    
                    
        virturalPosPro = dict()
        virturalNegPro = dict()
        virturalVocabulary = set(list(posVirtualFreq.keys()) + list(negVirtualFreq.keys()))


                                    
        for word in virturalVocabulary:
            freq = posVirtualFreq.get(word)
            if freq is None:
                freq = 0
            virturalPosPro.update({word : (lamda + freq) / (lamda * len(virturalVocabulary) + sum(posVirtualFreq.values())) })
        
        for word in virturalVocabulary:
            freq = negVirtualFreq.get(word)
            if freq is None:
                freq = 0
            virturalNegPro.update({word : (lamda + freq) / (lamda * len(virturalVocabulary) + sum(negVirtualFreq.values())) })
            
        
        predict = [calculateProba(review,positivePro,negativePro,posVirtualFreq,negVirtualFreq,virturalPosPro,virturalNegPro,len(virturalVocabulary)) for review in testData['Review']]
        result = np.zeros([len(predict),2])
        result = pd.DataFrame(result)
        result[0]=[1 if tem>0 else 0 for tem in predict]
        result[1]=list(testData['Label'])
        result[1] = [ 1 if (tem == 'POS') else 0 for tem in list(result[1])]
        target_names = ['class 0', 'class 1']      
        print(domainName, n, round(sklearn.metrics.f1_score(result[1],result[0], pos_label=0, average='binary'),4))
        f1Score += round(sklearn.metrics.f1_score(result[1],result[0], pos_label=1, average='binary'),4) 
        print(classification_report(result[1],result[0], target_names=target_names))
        n+=1
    scores[k] = f1Score/n
scores[-1] = scores[:-1]

Fold  0
..............................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................AlarmClock 0 0.5
             precision    recall  f1-score   support

    class 0       0.33      1.00      0.50        50
    class 1       1.00      0.23      0.38       130

avg / total       0.81      0.44      0.41       180

Fold  1
..................................